## Loading data

## K-fold cross-validation for number of trees

In [1]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

spambase_augmented = pd.read_csv('spambase_augmented.csv')

#Shuffle the data
spambase_augmented = spambase_augmented.sample(frac=1, random_state=42).reset_index(drop=True)

training_data, testing_data = train_test_split(spambase_augmented, test_size=0.2, random_state=42)


#Numbver of folds
k = 5

#Ensemble list of number of trees
ensemble = [50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300]

num_samples = training_data.shape[0]

features_folds = []

# Split the data into k folds
for i in range(k):
    features_folds.append(training_data.iloc[i * num_samples // k: (i + 1) * num_samples // k])


best_error = float('inf')
best_number_of_trees = ensemble[0]

for number_of_trees in ensemble:

    print("Number of trees:", number_of_trees)

    sum_error = 0
    average_error = 0

    for i in range(k):
        
        training_fold = pd.concat(features_folds[:i] + features_folds[i + 1:])
        testing_fold = features_folds[i]

        train_features = training_fold.drop(training_fold.columns[-1], axis=1)
        train_target = training_fold[training_fold.columns[-1]]

        test_features = testing_fold.drop(testing_fold.columns[-1], axis=1)
        test_target = testing_fold[testing_fold.columns[-1]]

        rf = RandomForestClassifier(n_estimators=number_of_trees, random_state=42, bootstrap=True, max_features='sqrt')

        rf.fit(train_features, train_target)

        predictions = rf.predict(test_features)

        accuracy = accuracy_score(test_target, predictions)

        error = 1 - accuracy

        sum_error = sum_error + error

    average_error = sum_error / (k)

    print("Average Error:", average_error)

    if average_error < best_error:
        print("New best error:", average_error)
        best_error = average_error
        best_number_of_trees = number_of_trees

print("Best number of trees:", best_number_of_trees)
print("Best error:", best_error)


Number of trees: 50
Average Error: 0.052989130434782615
New best error: 0.052989130434782615
Number of trees: 75
Average Error: 0.05326086956521736
Number of trees: 100
Average Error: 0.05326086956521738
Number of trees: 125
Average Error: 0.05353260869565217
Number of trees: 150
Average Error: 0.05353260869565217
Number of trees: 175
Average Error: 0.05271739130434783
New best error: 0.05271739130434783
Number of trees: 200
Average Error: 0.05271739130434783
Number of trees: 225
Average Error: 0.053804347826086986
Number of trees: 250
Average Error: 0.053804347826086965
Number of trees: 275
Average Error: 0.052989130434782594
Number of trees: 300
Average Error: 0.05353260869565215
Best number of trees: 175
Best error: 0.05271739130434783


In [7]:
errors = [
    0.052989130434782615,
    0.05326086956521736,
    0.05326086956521738,
    0.05353260869565217,
    0.05353260869565217,
    0.05271739130434783,
    0.05271739130434783,
    0.053804347826086986,
    0.053804347826086965,
    0.052989130434782594,
    0.05353260869565215
]

import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

def plot_accuracies_with_varying_number_of_trees(ensemble, errors, title):
    plt.figure(figsize=(10, 6))
    plt.plot(ensemble, errors, label='Average Error', linestyle='-', color='orange', marker='o', markersize=5)

    plt.title(title)
    plt.xlabel('Number of Trees')
    plt.ylabel('Average Error')
    plt.legend()

    plt.gca().yaxis.set_major_formatter(PercentFormatter(xmax=1))

    plt.grid(True)
    plt.show()


plot_accuracies_with_varying_number_of_trees(ensemble, errors, 'Average Error vs Number of Trees')


In [3]:
rf = RandomForestClassifier(n_estimators=best_number_of_trees, random_state=42, bootstrap=True, max_features='sqrt')
rf.fit(training_data.drop(training_data.columns[-1], axis=1), training_data[training_data.columns[-1]])


print("Best number of trees:", best_number_of_trees)
predictions = rf.predict(testing_data.drop(testing_data.columns[-1], axis=1))

accuracy = accuracy_score(testing_data[testing_data.columns[-1]], predictions)

error = 1 - accuracy

print("Error on testing data:", error)

Best number of trees: 175
Error on testing data: 0.05108695652173911
